In [1]:
import os
import sys
import math
import utils
import random
import layers
import network
import dataset
import binvox_rw
import numpy as np
import pandas as pd
from PIL import Image
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
# tf.contrib.eager.enable_eager_execution() 

In [2]:
network.setup_net()

(?, 24, 137, 137, 3)
(?, 24, 68, 68, 96)
(?, 24, 34, 34, 128)
(?, 24, 17, 17, 256)
(?, 24, 8, 8, 256)
(?, 24, 4, 4, 256)
(?, 24, 2, 2, 256)
(?, 24, 1024)
(4, 4, 4, ?, 24, 1024)
(4, 4, 4, ?, 256)
(?, 4, 4, 4, 256)
(?, 8, 8, 8, 256)
(?, 16, 16, 16, 128)
(?, 32, 32, 32, 64)
(?, 32, 32, 32, 32)
(?, 32, 32, 32, 2)


In [3]:
# setup dataset
shapenet=dataset.ShapeNet()
f=open("net.config")
shapenet.batch_size=int(f.readline())
data,label=shapenet.next_train_batch()
feed_dict={x:data, y: label};

NameError: name 'x' is not defined

In [ ]:
# debug
os.system("rm -rf out_dir")
print("debugging")
sess=tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
# tensorboard
writer = tf.summary.FileWriter("./logs/")
writer.add_graph(sess.graph)

In [ ]:

# enocder debug
out_root_dir="out_dir"
batch_size,time_steps,n_layers=shapenet.batch_size,24,7
for b in range(batch_size):
    for t in range(time_steps):
        out_dir=os.path.join(out_root_dir,str(b),str(t))
        if not os.path.isdir(out_dir):
            os.makedirs(out_dir)
        
        for i in range(n_layers):
            var=encoder_outputs[i]
            var=var[b,t]
            utils.imsave_multichannel(var.eval(feed_dict),out_dir+"/{}.png".format(str(i)))
        
        var=encoder_outputs[n_layers]
        var=var.eval(feed_dict)
        var=var[b,t]
        plt.figure()
        plt.plot(var)
        plt.savefig(out_dir+"/{}(feature_vector).png".format(str(n_layers)))

In [ ]:
# debug decoder net
out=tf.cast(tf.argmax(y_hat,axis=4),dtype=tf.float32)
out=sess.run(out,feed_dict=fd)
outvoxel=binvox_rw.Voxels(out,out.shape,[0,0,0],1,'xzy')
with open("out/cur_output.binvox",'w') as f:
    outvoxel.write(f)